In [1]:
import os 
import csv
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [2]:
cache_misses = pd.read_csv("../ce_data/caches/l1d_miss_ratio.csv")

In [3]:
reuse_distances = pd.DataFrame()
for file in os.listdir("../ce_data/reuse_distances"):
    x = pd.read_csv("../ce_data/reuse_distances/"+file)
    reuse_distances = reuse_distances.append(x, ignore_index=True)

<ipython-input-3-ff3ff2ea8081>:4: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  reuse_distances = reuse_distances.append(x, ignore_index=True)
<ipython-input-3-ff3ff2ea8081>:4: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  reuse_distances = reuse_distances.append(x, ignore_index=True)
<ipython-input-3-ff3ff2ea8081>:4: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  reuse_distances = reuse_distances.append(x, ignore_index=True)
<ipython-input-3-ff3ff2ea8081>:4: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  reuse_distances = reuse_distances.append(x, ignore_index=True)
<ipython-input-3-ff3ff2ea8081>:4: FutureWarning: The frame.append method

<ipython-input-3-ff3ff2ea8081>:4: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  reuse_distances = reuse_distances.append(x, ignore_index=True)
<ipython-input-3-ff3ff2ea8081>:4: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  reuse_distances = reuse_distances.append(x, ignore_index=True)
<ipython-input-3-ff3ff2ea8081>:4: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  reuse_distances = reuse_distances.append(x, ignore_index=True)
<ipython-input-3-ff3ff2ea8081>:4: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  reuse_distances = reuse_distances.append(x, ignore_index=True)
<ipython-input-3-ff3ff2ea8081>:4: FutureWarning: The frame.append method

<ipython-input-3-ff3ff2ea8081>:4: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  reuse_distances = reuse_distances.append(x, ignore_index=True)
<ipython-input-3-ff3ff2ea8081>:4: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  reuse_distances = reuse_distances.append(x, ignore_index=True)
<ipython-input-3-ff3ff2ea8081>:4: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  reuse_distances = reuse_distances.append(x, ignore_index=True)
<ipython-input-3-ff3ff2ea8081>:4: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  reuse_distances = reuse_distances.append(x, ignore_index=True)
<ipython-input-3-ff3ff2ea8081>:4: FutureWarning: The frame.append method

<ipython-input-3-ff3ff2ea8081>:4: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  reuse_distances = reuse_distances.append(x, ignore_index=True)
<ipython-input-3-ff3ff2ea8081>:4: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  reuse_distances = reuse_distances.append(x, ignore_index=True)
<ipython-input-3-ff3ff2ea8081>:4: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  reuse_distances = reuse_distances.append(x, ignore_index=True)
<ipython-input-3-ff3ff2ea8081>:4: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  reuse_distances = reuse_distances.append(x, ignore_index=True)
<ipython-input-3-ff3ff2ea8081>:4: FutureWarning: The frame.append method

<ipython-input-3-ff3ff2ea8081>:4: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  reuse_distances = reuse_distances.append(x, ignore_index=True)
<ipython-input-3-ff3ff2ea8081>:4: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  reuse_distances = reuse_distances.append(x, ignore_index=True)
<ipython-input-3-ff3ff2ea8081>:4: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  reuse_distances = reuse_distances.append(x, ignore_index=True)
<ipython-input-3-ff3ff2ea8081>:4: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  reuse_distances = reuse_distances.append(x, ignore_index=True)
<ipython-input-3-ff3ff2ea8081>:4: FutureWarning: The frame.append method

<ipython-input-3-ff3ff2ea8081>:4: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  reuse_distances = reuse_distances.append(x, ignore_index=True)
<ipython-input-3-ff3ff2ea8081>:4: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  reuse_distances = reuse_distances.append(x, ignore_index=True)
<ipython-input-3-ff3ff2ea8081>:4: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  reuse_distances = reuse_distances.append(x, ignore_index=True)
<ipython-input-3-ff3ff2ea8081>:4: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  reuse_distances = reuse_distances.append(x, ignore_index=True)
<ipython-input-3-ff3ff2ea8081>:4: FutureWarning: The frame.append method

In [4]:
print(cache_misses)
print(reuse_distances)

                         id        32        64       128       256       512  \
0       400.perlbench-41B_0  0.001166  0.001166  0.001166  0.001166  0.001166   
1       400.perlbench-41B_1  0.001646  0.001646  0.001646  0.001646  0.001646   
2       400.perlbench-41B_2  0.001015  0.001015  0.001015  0.001015  0.001015   
3       400.perlbench-41B_3  0.000598  0.000598  0.000598  0.000598  0.000598   
4       400.perlbench-41B_4  0.001044  0.001044  0.001044  0.001044  0.001044   
...                     ...       ...       ...       ...       ...       ...   
141556     657.xz_s-56B_744  0.005654  0.005654  0.005654  0.005654  0.005654   
141557     657.xz_s-56B_745  0.005642  0.005642  0.005642  0.005642  0.005642   
141558     657.xz_s-56B_746  0.005655  0.005655  0.005655  0.005655  0.005655   
141559     657.xz_s-56B_747  0.005641  0.005641  0.005641  0.005641  0.005641   
141560     657.xz_s-56B_748  0.005643  0.005643  0.005643  0.005643  0.005643   

            1024      2048 

In [5]:
cache_misses = cache_misses.sort_values("id", ascending=True, ignore_index=True)
reuse_distances = reuse_distances.sort_values("id", ascending=True, ignore_index=True)

In [6]:
# Quick check if all columns are the same
for i,j in zip(reuse_distances['id'], cache_misses['id']):
    if i!=j:
        print(i, j)

In [7]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(reuse_distances,cache_misses, test_size=0.3)

In [8]:
X_train_ids = X_train["id"]
X_test_ids = X_test["id"]
y_train_ids = y_train["id"]
y_test_ids = y_test["id"]

print(X_train_ids)

135350         654.roms_s-1613B_576
8281            410.bwaves-945B_136
108791     631.deepsjeng_s-928B_266
131404    649.fotonik3d_s-8225B_395
122485    648.exchange2_s-1511B_457
                    ...            
31006              445.gobmk-2B_366
107540          627.cam4_s-573B_489
69665        483.xalancbmk-736B_105
100289     623.xalancbmk_s-202B_703
138437          654.roms_s-842B_658
Name: id, Length: 99092, dtype: object


In [9]:
X_train = X_train.drop(columns=["id"])
X_test = X_test.drop(columns=["id"])
y_train = y_train.drop(columns=["id"])
y_test = y_test.drop(columns=["id"])

In [10]:
print(X_train.shape, X_test.shape)
print(y_train.shape, y_test.shape)

(99092, 896) (42469, 896)
(99092, 9) (42469, 9)


In [11]:
print(X_train.columns)

Index(['0', '1', '2', '3', '4', '5', '6', '7', '8', '9',
       ...
       '3623878656', '3690987520', '3758096384', '3825205248', '3892314112',
       '3959422976', '4026531840', '4093640704', '4160749568', '4227858432'],
      dtype='object', length=896)


In [50]:
from keras.models import Sequential
from keras.layers import Dense
from keras.metrics import MeanSquaredError
from keras.losses import MeanSquaredError
# Create the model
model = Sequential()

# Add the input layer and the first hidden layer
model.add(Dense(512, input_dim=896, activation='sigmoid'))

# Add the second hidden layer
model.add(Dense(512, activation='sigmoid'))

# Add the output layer
model.add(Dense(9, activation='sigmoid'))

# Compile the model
model.compile(loss=MeanSquaredError() , optimizer='adam')

# Print the model summary
model.summary()

Model: "sequential_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_15 (Dense)            (None, 512)               459264    
                                                                 
 dense_16 (Dense)            (None, 512)               262656    
                                                                 
 dense_17 (Dense)            (None, 9)                 4617      
                                                                 
Total params: 726,537
Trainable params: 726,537
Non-trainable params: 0
_________________________________________________________________


In [ ]:
model.fit(X_train.to_numpy(), y_train.to_numpy(), epochs=10)

Epoch 1/10
3097/3097 [==============================] - 18s 6ms/step - loss: 0.0042
Epoch 2/10
3097/3097 [==============================] - 18s 6ms/step - loss: 0.0019
Epoch 3/10
 689/3097 [=====>........................] - ETA: 13s - loss: 0.0017

In [48]:
f = model(X_test.to_numpy())

In [49]:
from sklearn.metrics import mean_squared_error
# Compare columns
for i,j in enumerate(['32','64','128','256','512','1024','2048','4096','8192']):
    model_out = f.numpy()[:,i]
    y_out = y_test.to_numpy()[:,i]
    print(f'Mean squared error for cache size {j}\t: {mean_squared_error(model_out, y_out)}')

Mean squared error for cache size 32	: 0.011600304637143219
Mean squared error for cache size 64	: 0.011689893597825407
Mean squared error for cache size 128	: 0.011640410262514857
Mean squared error for cache size 256	: 0.011613261505066488
Mean squared error for cache size 512	: 0.011635704829827806
Mean squared error for cache size 1024	: 0.011635562132036578
Mean squared error for cache size 2048	: 0.011632190480716027
Mean squared error for cache size 4096	: 0.011614503952587408
Mean squared error for cache size 8192	: 0.011554826958472562


In [41]:
print(f)

tf.Tensor(
[[0.07116638 0.07132334 0.07127499 ... 0.07131332 0.0706766  0.07000378]
 [0.07116638 0.07132334 0.07127499 ... 0.07131332 0.0706766  0.07000378]
 [0.07116638 0.07132334 0.07127499 ... 0.07131332 0.0706766  0.07000378]
 ...
 [0.07116638 0.07132334 0.07127499 ... 0.07131332 0.0706766  0.07000378]
 [0.07116638 0.07132334 0.07127499 ... 0.07131332 0.0706766  0.07000378]
 [0.07116626 0.07132336 0.07127503 ... 0.07131329 0.07067666 0.07000377]], shape=(42469, 9), dtype=float32)


In [42]:
print(y_test.to_numpy(), 'shape =', y_test.to_numpy().shape)

[[0.00049033 0.00049033 0.00049033 ... 0.00049031 0.0004903  0.0004903 ]
 [0.03452206 0.03452162 0.03447242 ... 0.03378608 0.03379391 0.03377272]
 [0.05702127 0.05708875 0.0570729  ... 0.0570464  0.05692536 0.05498693]
 ...
 [0.45148744 0.45162288 0.45165131 ... 0.47003715 0.47029839 0.47030101]
 [0.02505576 0.02505067 0.02505026 ... 0.02502404 0.02495715 0.02493541]
 [0.05687171 0.05687171 0.05687047 ... 0.05536013 0.05535908 0.05535908]] shape = (42469, 9)


In [ ]:
print(np.sum(model(X_train.to_numpy()),axis=0))